In [2]:
#  api키  7a1258842909fa315bd1a319017098ca

import json
import requests
movies = []
# genres = [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 16, "name": "Animation"}, {"id": 35, "name": "Comedy"}, {"id": 80, "name": "Crime"}, {"id": 99, "name": "Documentary"}, {"id": 18, "name": "Drama"}, {"id": 10751, "name": "Family"}, {"id": 14, "name": "Fantasy"}, {"id": 36, "name": "History"}, {
#     "id": 27, "name": "Horror"}, {"id": 10402, "name": "Music"}, {"id": 9648, "name": "Mystery"}, {"id": 10749, "name": "Romance"}, {"id": 878, "name": "Science Fiction"}, {"id": 10770, "name": "TV Movie"}, {"id": 53, "name": "Thriller"}, {"id": 10752, "name": "War"}, {"id": 37, "name": "Western"}]

# for genre in genres:
#     temp = {}
#     temp['model'] = 'movies.genre'
#     temp['pk'] = genre['id']
#     fields = {}
#     fields['name'] = genre['name']
#     temp['fields'] = fields
#     movies.append(temp)


# for i in range(1, 20):
#     data = requests.get(
#         f'https://api.themoviedb.org/3/movie/popular?api_key=245ecc0e095645bb9a67ccf2d25856ec&language=ko&region=KR&page={i}').json()
#     data_result = data['results']
#     for result in data_result:
#         temp = {}
#         temp['model'] = 'movies.movie'
#         field = {}
#         movie_field = ['title','id', 'release_date', 'popularity', 'vote_count',
#                        'vote_average', 'overview', 'poster_path', 'genre_ids']

#         for key, value in result.items():
#             if key in movie_field and key == 'genre_ids':
#                 field['genres'] = value
#             elif key in movie_field and key == 'poster_path':
#                 field['poster_path'] = f'https://image.tmdb.org/t/p/w500/{value}'
#             elif key in movie_field:
#                 field[key] = value

#         temp['fields'] = field
#         movies.append(temp)

# with open('movies_now.json', 'w', encoding="UTF-8") as make_file:
#     json.dump(movies, make_file, indent='\t',ensure_ascii=False)


In [3]:
class URLMaker:
    url = 'https://api.themoviedb.org/3'

    def __init__(self, key):
        self.key = key

    def get_movie_url(self, category='movie', feature='popular', page='1'):
        url = f'{self.url}/{category}/{feature}'
        url += f'?api_key={self.key}&language=ko&page={str(page)}'

        return url

    def movie_id(self, title):
        url = self.get_url('search', 'movie', region='KR', language='ko', query=title)
        res = requests.get(url)
        movie = res.json()

        if len(movie.get('results')):
            return movie.get('results')[0].get('id')
        else:
            return None

    def get_genre_url(self):
        url = f'{self.url}/genre/movie/list?api_key={self.key}&language=ko'
        return url

TMDB_KEY = '7a1258842909fa315bd1a319017098ca'
url = URLMaker(TMDB_KEY)

def create_genre_data():
    genre_url = url.get_genre_url()
    raw_data = requests.get(genre_url)
    json_data = raw_data.json()
    genres = json_data.get('genres')

    genre_data = []

    for genre in genres:
        tmp = {
            'model': 'movies.genre',
            'pk': genre['id'],
            'fields': {
                'name': genre['name']
            }
        }
        genre_data.append(tmp)

    with open('tmdb.json', 'w') as f:
        json.dump(genre_data, f, indent=4,ensure_ascii=False)

def create_movie_data():
    with open('tmdb.json', 'r+') as f:
        movie_data = json.load(f)

    for page in range(1, 501):
        raw_data = requests.get(url.get_movie_url(page=page))
        json_data = raw_data.json()
        movies = json_data.get('results')

        for movie in movies:
            if movie.get('release_date') == "" or movie.get('poster_path') == "":
                continue

            movie.pop('backdrop_path')
            movie.pop('video')
            movie['like_users'] = []
            tmp = {
                'model': 'movies.movie',
                'pk': movie.pop('id'),
                'fields': movie,
            }
            movie_data.append(tmp)

    with open('tmdbKr.json', 'w', encoding="UTF-8") as f:
        json.dump(movie_data, f, indent=4,ensure_ascii=False)

create_genre_data()
create_movie_data()

### DB에 넣기

In [24]:
# movie_data = open('movies_now.json',encoding='UTF8')
# data = json.load(movie_data)

with open('파일이름.json', encoding="UTF-8") as file:
    movie = json.load(file)
  

In [38]:
import pymysql

conn = pymysql.connect(host='localhost',
                       user='root',
                       password='root',
                       port=3306,
                       db='mv',
                       charset='utf8')
database_name = 'movie_info'
cursor = conn.cursor()

In [ ]:
# 장르가 여러개 들어가는 데이터가 있어서 안들어감(몽고db 가능)

for movies in movie:
    if movies['model'] == 'movies.movie':
        title = movies['fields']['title']
        genre = movies['fields']['genres']
        vote = movies['fields']['vote_average']
        print(title, genre, vote)
        sql = 'insert into ' + database_name + '(title, genres, vote_average) values (%s, %s, %s)' 

        cursor.execute(sql, (title, genre, vote))
conn.commit()

In [ ]:
# 원하는 데이터만 뽑아서 넣기

movies = list()

for data_ in data:
    if data_['model'] == 'movies.movie':
        db['title_table'].insert(data_['fields']['title'])